Create files with monthly time series of mooring data in the Central and Dotson trough

Need these to plot the mean direction of the flow in the location plot

In [1]:
import numpy as np
from numpy import ma

import xarray as xr
import pandas as pd

import matplotlib.dates as mdates
import matplotlib as mpl

import sys
import gsw

In [2]:
#-------------------------------------------------------------------
# Directories
#-------------------------------------------------------------------
voldir = '/Volumes/SamT5/PhD/data/'
mdir = voldir + 'moorings/dragomir_phd/'

localdir = '/Users/ocd1n16/PhD_git/'

auxscriptdir = localdir + 'aux_func/'
sys.path.append(auxscriptdir)
import aux_func_trend as fc

### DT

In [3]:
# time end points
s1_start = '2010-03-01'
s1_end = '2015-12-30'

In [4]:
# S1 mooring data file
# 8 m grid
mfile = 's1_raw_uv_hourly_adcp_404m_540m_2010_2016.nc'

# depth range (412, 528)
dep_min_max = [412, 528]

with xr.open_dataset(mdir + mfile) as s1moor:
    print(s1moor.keys())

s1 = s1moor.sel(depth=slice(dep_min_max[0], dep_min_max[1]))

# average data daily
s1d = s1.resample(time="1D").mean()

# projection on rotated reference frame
tim, dep = s1d.u.shape
rot_u, rot_v = [np.ones((tim, dep)) for _ in range(2)]

for i in range(dep):
    rot_vel = fc.rotate_frame(s1d.u.isel(depth=i), s1d.v.isel(depth=i), 51, 'clockwise')
    rot_u[:, i] = rot_vel.u.values
    rot_v[:, i] = rot_vel.v.values
    
# add new components to existing dataset
s1d["u_rot"] = (("time", 'depth'), rot_u)
s1d["v_rot"] = (("time", "depth"), rot_v)

# average with depth + change units to cm/s to match CT
ts1d = s1d.mean("depth") * 1e2

KeysView(<xarray.Dataset>
Dimensions:  (time: 51839, depth: 18)
Coordinates:
  * time     (time) datetime64[ns] 2010-02-15T15:00:00 ... 2016-01-17T03:00:00
  * depth    (depth) float64 404.0 412.0 420.0 428.0 ... 516.0 524.0 532.0 540.0
Data variables:
    u        (time, depth) float64 ...
    v        (time, depth) float64 ...
Attributes:
    lon:      -116.358
    lat:      -72.468)


In [6]:
ts1d.attrs['description'] = "Daily time means of depth-averaged (412-528m) velocity components at S1."\
"_rot = velocity components are projected onto a 42 deg clockwise rotated frame."
ts1d.attrs["units"] = 'cm/s'
ts1d.to_netcdf(mdir + "daily_dt_uv.nc")

In [7]:
# monthly averages
mts1 = ts1d.resample(time="1MS").mean()
mts1 = mts1.sel(time=slice(s1_start, s1_end))

In [8]:
mts1.attrs['description'] = "Monthly time means of depth-averaged (412-528m) "\
    "velocity components at S1. "\
    "_rot = velocity components are projected onto a 42 deg clockwise rotated frame."

mts1.to_netcdf(mdir + "monthly_dt_uv.nc")

### CT

In [9]:
#------------------------------------------------------------------
#            ~ ~ ~    OTHER MOORINGS     ~ ~ ~   
#------------------------------------------------------------------
# hourly
#bsr12_0 = xr.open_dataset(mdir_pd + 'bsr12_0_raw.nc') # 400 m
bsr12_1 = xr.open_dataset(mdir + 'bsr12_1_raw.nc') # 553 m
bsr13 = xr.open_dataset(mdir + 'bsr13a_raw.nc') # 338 m

istar1 = xr.open_dataset(mdir + 'istar1_raw.nc') # 581 m

troughW0 = xr.open_dataset(mdir + 'troughW0_raw.nc') # 423 m
troughW1 = xr.open_dataset(mdir + 'troughW1_raw.nc') # 555 m

troughE0 = xr.open_dataset(mdir + 'troughE0_raw.nc') # 467 m
troughE1 = xr.open_dataset(mdir + 'troughE1_raw.nc') # 595 m

# daily means
bsr12_d = bsr12_1.resample(time='1D').mean()
bsr13_d = bsr13.resample(time='1D').mean()
istar1_d = istar1.resample(time='1D').mean()
troughW1_d = troughW1.resample(time='1D').mean()
troughE1_d = troughE1.resample(time='1D').mean()

# vel components projected onto rotated frame; daily averages
bsr12_rot = fc.rotate_frame(bsr12_d.u, bsr12_d.v, 56, 'clockwise')
bsr13_rot = fc.rotate_frame(bsr13_d.u, bsr13_d.v, 17, 'anticlockwise')
istar1_rot = fc.rotate_frame(istar1_d.u, istar1_d.v, 47, 'clockwise')
troughW1_rot = fc.rotate_frame(troughW1_d.u, troughW1_d.v, 52, 'clockwise')
#troughE1_rot = fc.rotate_frame(troughE1_d.u, troughE1_d.v, 26, 'clockwise')

In [10]:
bsr12_d["u_rot"] = ("time", bsr12_rot.u.values)
bsr12_d["v_rot"] = ("time", bsr12_rot.v.values)

bsr13_d["u_rot"] = ("time", bsr13_rot.u.values)
bsr13_d["v_rot"] = ("time", bsr13_rot.v.values)

istar1_d["u_rot"] = ("time", istar1_rot.u.values)
istar1_d["v_rot"] = ("time", istar1_rot.v.values)

troughW1_d["u_rot"] = ("time", troughW1_rot.u.values)
troughW1_d["v_rot"] = ("time", troughW1_rot.v.values)

In [11]:
bsr12_d.attrs['description'] = 'Daily time means of velocity components of BSR12 at 553 m. rotation angle=54c'
bsr13_d.attrs['description'] = 'Daily time means of velocity components of BSR13a at 338 m. rotation angle=32a'
istar1_d.attrs['description'] = 'Daily time means of velocity components of istar1 at 581 m. rotation angle=46c'
troughW1_d.attrs['description'] = 'Daily time means of velocity components of troughW at 555 m. rotation angle=53c'

bsr12_d.to_netcdf(mdir + 'daily_bsr12_uv.nc')
bsr13_d.to_netcdf(mdir + 'daily_bsr13_uv.nc')
istar1_d.to_netcdf(mdir + 'daily_istar1_uv.nc')
troughW1_d.to_netcdf(mdir + 'daily_troughW_uv.nc')

### Monthly averages

In [12]:
# monthly averages
bsr12_m = bsr12_d.resample(time='1MS').mean().sel(time=slice('2009-03-01', '2010-12-01'))
bsr13_m = bsr13_d.resample(time='1MS').mean().sel(time=slice('2011-01-01', '2012-02-01'))
istar1_m = istar1_d.resample(time='1MS').mean().sel(time=slice('2012-03-01', '2014-02-01'))
troughW_m = troughW1_d.resample(time='1MS').mean().sel(time=slice('2014-03-01', '2016-01-01'))

In [13]:
bsr12_m.attrs['description'] = 'Monthly time means of velocity components of BSR12 at 553 m. rotation angle=54c'
bsr13_m.attrs['description'] = 'Monthly time means of velocity components of BSR13a at 338 m. rotation angle=32a'
istar1_m.attrs['description'] = 'Monthly time means of velocity components of istar1 at 581 m. rotation angle=46c'
troughW_m.attrs['description'] = 'Monthly time means of velocity components of troughW at 555 m. rotation angle=53c'

bsr12_m.to_netcdf(mdir + 'monthly_bsr12_uv.nc')
bsr13_m.to_netcdf(mdir + 'monthly_bsr13_uv.nc')
istar1_m.to_netcdf(mdir + 'monthly_istar1_uv.nc')
troughW_m.to_netcdf(mdir + 'monthly_troughW_uv.nc')